# Evaluation of the novel methods

In [ ]:
ds_names = [
    "2-EMOT",
    "3-SCENE",
    "4-FLAGS",
    "5-FOODTRUCK",
    "6-YEAST",
    "7-BIRDS",
    "8-GENBASE",
    "9-MEDC",
    "10-ENRON",
    "11-MEDIAMILL",
    ]

In [ ]:
import pandas as pd
def get_dataset(ds: str):
    path_X = f"./data/datasets/{ds}_X.csv"
    path_y = f"./data/datasets/{ds}_y.csv"
    X = pd.read_csv(path_X)
    y = pd.read_csv(path_y)
    return X, y

X, y = get_dataset("3-SCENE")

X, y

In [ ]:
from sklearn.multioutput import ClassifierChain
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import RepeatedKFold
import numpy as np
from sklearn.metrics import hamming_loss, zero_one_loss 

from mcts_inference.brute_force import brute_force as bf
from mcts_inference.mcts import MCTS
from mcts_inference.mc import MCC
from mcts_inference.policy import UCB
from mcts_inference.constraints import Constraint
from mcts_inference.mcts_config import MCTSConfig

def losses(X_train, y_train, X_test, y_test, chain, algo, config, loss_fn=hamming_loss):
    chain = chain.fit(X_train, y_train)
    if algo is None:
        y_pred = chain.predict(X_test)
    elif config is not None:
        y_pred = np.abs(algo(X_test, chain, config))
    else:
        raise ValueError("Config cannot be None if algo is not None")

    return loss_fn(y_test, y_pred)

def loss_algos(ds, k=5, n_repeats = 1, random_state=0, loss_fn=hamming_loss):
    X, y = get_dataset(ds)
    chain = ClassifierChain(LogisticRegression(solver="liblinear", max_iter=10000))

    n_iter = 100
    d_time = 1.
    
    bf_loss = []
    cc_loss = []
    mc_loss = []
    mcts_loss = []
    mct1s_loss = []

    rkf = RepeatedKFold(n_splits=k, n_repeats=n_repeats, random_state=random_state)
    for train_idx, test_idx in rkf.split(X, y):
        # print(i)
        X_train, X_test = X.iloc[train_idx], X.iloc[test_idx]
        y_train, y_test = y.iloc[train_idx], y.iloc[test_idx]

        config = MCTSConfig(n_classes=y.shape[1], constraint=Constraint(time=True, d_time=d_time))
        loss = losses(X_train, y_train, X_test, y_test, chain, algo=bf, config=config, loss_fn=loss_fn)
        bf_loss.append(loss)

        loss = losses(X_train, y_train, X_test, y_test, chain, algo=None, config=None, loss_fn=loss_fn)
        cc_loss.append(loss)

        config = MCTSConfig(n_classes=y.shape[1], selection_policy=UCB(2), constraint=Constraint(max_iter=True, n_iter=n_iter))
        loss = losses(X_train, y_train, X_test, y_test, chain, MCC, config, loss_fn=loss_fn)
        mc_loss.append(loss)

        config = MCTSConfig(n_classes=y.shape[1], selection_policy=UCB(2), constraint=Constraint(max_iter=True, n_iter=n_iter))
        loss = losses(X_train, y_train, X_test, y_test, chain, MCTS, config, loss_fn=loss_fn)
        mcts_loss.append(loss)

        config = MCTSConfig(n_classes=y.shape[1], selection_policy=UCB(2), constraint=Constraint(max_iter=True, n_iter=n_iter), step_once=False)
        loss = losses(X_train, y_train, X_test, y_test, chain, MCTS, config, loss_fn=loss_fn)
        mct1s_loss.append(loss)

    return {"BF": np.mean(bf_loss), "CC": np.mean(cc_loss), "MC": np.mean(mc_loss), "MCTS": np.mean(mcts_loss), "MCTS1S": np.mean(mct1s_loss)}

In [ ]:
from sklearn.metrics import hamming_loss

dataset_hamming_losses = dict()
print("- Hamming Loss -")
for ds in ds_names:
    print(f"> Dataset: {ds}")
    try:
        L = loss_algos(ds, k=5, n_repeats=1, random_state=0, loss_fn=hamming_loss)
        print(f"\tBrute Force: {L['BF']}, Score: {1-L['BF']}")
        print(f"\tClassifier Chain (CC): {L['CC']}, Score: {1-L['CC']}")
        print(f"\tMonte Carlo CC: {L['MC']}, Score: {1-L['MC']}")
        print(f"\tMCTS: {L['MCTS']}, Score: {1-L['MCTS']}")
        print(f"\tMCTS1S: {L['MCTS1S']}, Score: {1-L['MCTS1S']}")
        dataset_hamming_losses[ds] = L
    except Exception as e:
        print(f"Error in {ds}: {e}")

In [ ]:
from sklearn.metrics import zero_one_loss

dataset_zero_one_losses = dict()
print("- Zero-One Loss -")
for ds in ds_names:
    print(f"> Dataset: {ds}")
    try:
        L = loss_algos(ds, k=5, n_repeats=1, random_state=0, loss_fn=zero_one_loss)
        print(f"\tBrute Force: {L['BF']}, Score: {1-L['BF']}")
        print(f"\tClassifier Chain (CC): {L['CC']}, Score: {1-L['CC']}")
        print(f"\tMonte Carlo CC: {L['MC']}, Score: {1-L['MC']}")
        print(f"\tMCTS: {L['MCTS']}, Score: {1-L['MCTS']}")
        print(f"\tMCTS1S: {L['MCTS1S']}, Score: {1-L['MCTS1S']}")
        dataset_zero_one_losses[ds] = L
    except Exception as e:
        print(f"Error in {ds}: {e}")